In [31]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
from glob import glob
import seaborn as sns

In [33]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

In [34]:
class LeNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model
		model = Sequential()
		inputShape = (height, width, depth)
		# if we are using "channels first", update the input shape
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
   # first set of CONV => RELU => POOL layers
		model.add(Conv2D(64, (3, 3), padding="same",input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  	# second set of CONV => RELU => POOL layers
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(64))
		model.add(Activation("relu"))
		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))
		# return the constructed network architecture
		return model

  



In [35]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

In [36]:
data_dir = '/content/gdrive/MyDrive/'

In [37]:
train_dir = data_dir + 'MURA-v1.1/train' # Path to train directory
val_dir = data_dir + 'MURA-v1.1/valid' # Path to validation directory

In [49]:
train_imgs_path=pd.read_csv(data_dir+'MURA-v1.1/train_image_paths.csv', names=['Train_Image'])
train_labels = pd.read_csv(data_dir+'MURA-v1.1/train_labeled_studies.csv', names=['Train_Image','Train_Label'])
test_imgs_path = pd.read_csv(data_dir+'MURA-v1.1/valid_image_paths.csv' ,  names=['Test_Image'])
test_labels = pd.read_csv(data_dir+'MURA-v1.1/valid_labeled_studies.csv', names=['Test_Image','Test_Label'])

In [39]:
test_labels_df = pd.read_csv(data_dir+'MURA-v1.1/valid_labeled_studies.csv', names=['Test_Image','Test_Label'])
train_labels_df = pd.read_csv(data_dir+'MURA-v1.1/train_labeled_studies.csv', names=['Train_Image','Train_Label'])

In [40]:
train_image_df = pd.read_csv(data_dir+'MURA-v1.1/train_image_paths.csv', names=['Train_Image'])
train_image_df["Train_Image"] = data_dir + train_image_df["Train_Image"]
train_image_df.head()

,Train_Image
0,/content/gdrive/MyDrive/MURA-v1.1/train/XR_HAN...
1,/content/gdrive/MyDrive/MURA-v1.1/train/XR_HAN...
2,/content/gdrive/MyDrive/MURA-v1.1/train/XR_HAN...
3,/content/gdrive/MyDrive/MURA-v1.1/train/XR_HAN...
4,/content/gdrive/MyDrive/MURA-v1.1/train/XR_HAN...


In [41]:
test_image_df = pd.read_csv(data_dir+'MURA-v1.1/valid_image_paths.csv', names=['Test_Image'])
test_image_df["Test_Image"] = data_dir + test_image_df["Test_Image"]
test_image_df.head()

,Test_Image
0,/content/gdrive/MyDrive/MURA-v1.1/valid/XR_HAN...
1,/content/gdrive/MyDrive/MURA-v1.1/valid/XR_HAN...
2,/content/gdrive/MyDrive/MURA-v1.1/valid/XR_HAN...
3,/content/gdrive/MyDrive/MURA-v1.1/valid/XR_HAN...
4,/content/gdrive/MyDrive/MURA-v1.1/valid/XR_HAN...


In [51]:
train_labels['Body_Part']=train_labels['Train_Image'].apply(lambda x: str(x.split('/')[2])[3:])
train_labels['Study_Type']=train_labels['Train_Image'].apply(lambda x: str(x.split('/')[4])[:6])
test_labels['Body_Part']=test_labels['Test_Image'].apply(lambda x: str(x.split('/')[2])[3:])
test_labels['Study_Type']=test_labels['Test_Image'].apply(lambda x: str(x.split('/')[4])[:6])

In [55]:
import seaborn as sns


In [56]:
sns.countplot(train_labels['Train_Label'])

In [57]:
plt.figure(figsize=(15,7))
sns.countplot(data=train_labels,x='Body_Part',hue='Train_Label')

In [58]:
plt.figure(figsize=(15,7))
sns.countplot(data=train_labels,x='Study_Type',hue='Train_Label')

In [ ]:
EPOCHS = 250
INIT_LR = 1e-3
BS = 32
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

[INFO] loading images...


In [ ]:
# grab the image paths and randomly shuffle them
imagePaths = train_imgs_path.values
random.seed(42)
random.shuffle(imagePaths)
print(imagePaths)

[['MURA-v1.1/train/XR_HAND/patient09734/study1_positive/image1.png']
 ['MURA-v1.1/train/XR_HAND/patient09734/study1_positive/image1.png']
 ['MURA-v1.1/train/XR_HAND/patient09734/study1_positive/image3.png']
 ...
 ['MURA-v1.1/train/XR_HAND/patient09787/study1_positive/image1.png']
 ['MURA-v1.1/train/XR_HAND/patient09962/study1_positive/image3.png']
 ['MURA-v1.1/train/XR_HAND/patient11096/study1_negative/image2.png']]


In [ ]:
for imagePath in train_imgs_path.values:
  	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(data_dir + imagePath[0])
	image = cv2.resize(image, (150, 150))
	image = img_to_array(image)
	data.append(image)

In [ ]:
trainX = data

In [ ]:
names=train_image_df['Train_Image'].values
labels =[]

for i in names:
    if ('positive' in i):
        labels.append('1')
    elif('negative' in i):
        labels.append('0')


In [ ]:
trainY = labels
print(trainY)

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',

In [ ]:
datatest = []
labletest = []

In [ ]:
for imagePath in test_imgs_path.values:
  image = cv2.imread(data_dir + imagePath[0])
  image = cv2.resize(image, (150, 150))
  image = img_to_array(image)
  datatest.append(image)


In [ ]:
testX = datatest

In [ ]:
names=test_image_df['Test_Image'].values
labletest =[]

for i in names:
    if ('positive' in i):
        labletest.append('1')
    elif('negative' in i):
        labletest.append('0')

In [ ]:
testY = labletest
print(testY)

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0',

In [ ]:
# scale the raw pixel intensities to the range [0, 1]
trainX = np.array(trainX, dtype="float") / 255.0
testX = np.array(testX, dtype="float") / 255.0
trainY = np.array(trainY)
testY = np.array(testY)
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

In [ ]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [ ]:
import tensorflow as tf
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='/content/gdrive/MyDrive/MURA-v1.1/newxray212.h5', 
                                                    monitor = 'accuracy',
                                                    save_weights_only = False,
                                                    save_best_only=True)

In [ ]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=150, height=150, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the network
print("[INFO] training network...")
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS,callbacks = checkpoint_cb, verbose=1)
# save the model to disk
print("[INFO] serializing network...")

[INFO] compiling model...
[INFO] training network...
Epoch 1/250
173/173 [==============================] - 48s 203ms/step - loss: 0.5926 - accuracy: 0.7320 - val_loss: 0.7473 - val_accuracy: 0.5891
Epoch 2/250
173/173 [==============================] - 35s 201ms/step - loss: 0.5841 - accuracy: 0.7322 - val_loss: 0.7305 - val_accuracy: 0.5891
Epoch 3/250
173/173 [==============================] - 35s 199ms/step - loss: 0.5825 - accuracy: 0.7331 - val_loss: 0.7519 - val_accuracy: 0.5891
Epoch 4/250
173/173 [==============================] - 34s 196ms/step - loss: 0.5839 - accuracy: 0.7324 - val_loss: 0.7017 - val_accuracy: 0.5891
Epoch 5/250
173/173 [==============================] - 34s 195ms/step - loss: 0.5836 - accuracy: 0.7318 - val_loss: 0.7215 - val_accuracy: 0.5891
Epoch 6/250
173/173 [==============================] - 34s 195ms/step - loss: 0.5834 - accuracy: 0.7320 - val_loss: 0.7320 - val_accuracy: 0.5891
Epoch 7/250
173/173 [==============================] - 34s 195ms/step -

In [ ]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Santa/Not Santa")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
